In [1]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
import nltk
from sklearn.metrics import f1_score

data_dir = 'data'
ckpt_dir = 'checkpoint'
word_embd_dir = 'checkpoint/word_embd'
model_dir = 'checkpoint/model2v2'

word_embd_dim = 100
pos_embd_dim = 25
dep_embd_dim = 25
word_vocab_size = 400001
pos_vocab_size = 10
dep_vocab_size = 21
relation_classes = 19
state_size = 100
batch_size = 10
channels = 3
lambda_l2 = 0.0001
max_len_path = 70
starter_learning_rate = 0.001
decay_steps = 2000
decay_rate = 0.96

In [2]:
with tf.name_scope("input"):
    path_length = tf.placeholder(tf.int32, shape=[batch_size], name="path1_length")
    word_ids = tf.placeholder(tf.int32, shape=[batch_size, max_len_path], name="word_ids")
    pos_ids = tf.placeholder(tf.int32, [batch_size, max_len_path], name="pos_ids")
    dep_ids = tf.placeholder(tf.int32, [batch_size, max_len_path], name="dep_ids")
    y = tf.placeholder(tf.int32, [batch_size], name="y")

with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embedded_word = tf.nn.embedding_lookup(W, word_ids)
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

with tf.name_scope("pos_embedding"):
    W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
    embedded_pos = tf.nn.embedding_lookup(W, pos_ids)
    pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

with tf.name_scope("dep_embedding"):
    W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
    embedded_dep = tf.nn.embedding_lookup(W, dep_ids)
    dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})

In [3]:
embedded_word_rev = tf.reverse(embedded_word, [1])
embedded_pos_rev = tf.reverse(embedded_pos, [1])
embedded_dep_rev = tf.reverse(embedded_dep, [1])
path_length_rev = tf.reverse(path_length, [0])

In [4]:
with tf.variable_scope("word_lstm_fw"):
    cell = tf.contrib.rnn.BasicLSTMCell(state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word, sequence_length=path_length, dtype=tf.float32)
    state_series_word_fw = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("pos_lstm_fw"):
    cell = tf.contrib.rnn.BasicLSTMCell(state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos, sequence_length=path_length, dtype=tf.float32)
    state_series_pos_fw = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("dep_lstm_fw"):
    cell = tf.contrib.rnn.BasicLSTMCell(state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep, sequence_length=path_length, dtype=tf.float32)
    state_series_dep_fw = tf.reduce_max(state_series, axis=1)
    
with tf.variable_scope("word_lstm_bw"):
    cell = tf.contrib.rnn.BasicLSTMCell(state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word_rev, sequence_length=path_length_rev, dtype=tf.float32)
    state_series_word_bw = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("pos_lstm_bw"):
    cell = tf.contrib.rnn.BasicLSTMCell(state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos_rev, sequence_length=path_length_rev, dtype=tf.float32)
    state_series_pos_bw = tf.reduce_max(state_series, axis=1)

with tf.variable_scope("dep_lstm_bw"):
    cell = tf.contrib.rnn.BasicLSTMCell(state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep_rev, sequence_length=path_length_rev, dtype=tf.float32)
    state_series_dep_bw = tf.reduce_max(state_series, axis=1)
    
state_series = tf.concat([state_series_word_fw, state_series_pos_fw, state_series_dep_fw, state_series_word_bw, state_series_pos_bw, state_series_dep_bw], 1)


In [5]:
state_series

<tf.Tensor 'concat:0' shape=(10, 600) dtype=float32>

In [6]:
with tf.name_scope("hidden_layer"):
    W = tf.Variable(tf.truncated_normal([600, 100], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([100]), name="b")
    y_hidden_layer = tf.matmul(state_series, W) + b

with tf.name_scope("dropout"):
    y_hidden_layer_drop = tf.nn.dropout(y_hidden_layer, 0.3)

with tf.name_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([100, relation_classes], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([relation_classes]), name="b")
    logits = tf.matmul(y_hidden_layer_drop, W) + b
    predictions = tf.argmax(logits, 1)

tv_all = tf.trainable_variables()
tv_regu = []
non_reg = ["word_embedding/W:0","pos_embedding/W:0",'dep_embedding/W:0',"global_step:0",'hidden_layer/b:0','softmax_layer/b:0']
for t in tv_all:
    if t.name not in non_reg:
        if(t.name.find('biases')==-1):
            tv_regu.append(t)

with tf.name_scope("loss"):
    l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    total_loss = loss + l2_loss

global_step = tf.Variable(0, trainable=False, name="global_step")

learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, decay_steps, decay_rate, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(total_loss, global_step=global_step)

In [7]:
f = open('data/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"
word2id[unknown_token] = word_vocab_size -1
id2word[word_vocab_size-1] = unknown_token

pos_tags_vocab = []
for line in open('data/pos_tags.txt'):
        pos_tags_vocab.append(line.strip())

dep_vocab = []
for line in open('data/dependency_types.txt'):
    dep_vocab.append(line.strip())

relation_vocab = []
for line in open('data/relation_types.txt'):
    relation_vocab.append(line.strip())

rel2id = dict((w, i) for i,w in enumerate(relation_vocab))
id2rel = dict((i, w) for i,w in enumerate(relation_vocab))

pos_tag2id = dict((w, i) for i,w in enumerate(pos_tags_vocab))
id2pos_tag = dict((i, w) for i,w in enumerate(pos_tags_vocab))

dep2id = dict((w, i) for i,w in enumerate(dep_vocab))
id2dep = dict((i, w) for i,w in enumerate(dep_vocab))

pos_tag2id['OTH'] = 9
id2pos_tag[9] = 'OTH'

dep2id['OTH'] = 20
id2dep[20] = 'OTH'

JJ_pos_tags = ['JJ', 'JJR', 'JJS']
NN_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']
RB_pos_tags = ['RB', 'RBR', 'RBS']
PRP_pos_tags = ['PRP', 'PRP$']
VB_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
_pos_tags = ['CC', 'CD', 'DT', 'IN']

def pos_tag(x):
    if x in JJ_pos_tags:
        return pos_tag2id['JJ']
    if x in NN_pos_tags:
        return pos_tag2id['NN']
    if x in RB_pos_tags:
        return pos_tag2id['RB']
    if x in PRP_pos_tags:
        return pos_tag2id['PRP']
    if x in VB_pos_tags:
        return pos_tag2id['VB']
    if x in _pos_tags:
        return pos_tag2id[x]
    else:
        return 9

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [9]:
# f = open('data/word_embedding', 'rb')
# word_embedding = pickle.load(f)
# f.close()
# sess.run(embedding_init, feed_dict={embedding_placeholder:word_embedding})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')

In [10]:
# model = tf.train.latest_checkpoint(model_dir)
# saver.restore(sess, model)

In [11]:
latest_embd = tf.train.latest_checkpoint(word_embd_dir)
word_embedding_saver.restore(sess, latest_embd)

INFO:tensorflow:Restoring parameters from checkpoint/word_embd/word_embd


In [12]:
f = open('data/train_lca_paths', 'rb')
word_p, dep_p, pos_p = pickle.load(f)
f.close()
relations = []
for line in open('data/train_relations.txt'):
    relations.append(line.strip().split()[1])

length = len(word_p)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p[i]):
        word = word.lower()
        word_p[i][j] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p[i]):
        dep_p[i][l] = d if d in dep2id else 'OTH'
        
word_p_ids = np.ones([length, max_len_path],dtype=int)
pos_p_ids = np.ones([length, max_len_path],dtype=int)
dep_p_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path_len = np.array([len(w) for w in word_p], dtype=int)

for i in range(length):
    for j, w in enumerate(word_p[i]):
        word_p_ids[i][j] = word2id[w]
        
    for j, w in enumerate(pos_p[i]):
        pos_p_ids[i][j] = pos_tag(w)
        
    for j, w in enumerate(dep_p[i]):
        dep_p_ids[i][j] = dep2id[w]

In [13]:
num_epochs = 10
for i in range(num_epochs):
    loss_per_epoch = 0
    for j in range(num_batches):
        feed_dict = {
            path_length:path_len[j*batch_size:(j+1)*batch_size],
            word_ids:word_p_ids[j*batch_size:(j+1)*batch_size],
            pos_ids:pos_p_ids[j*batch_size:(j+1)*batch_size],
            dep_ids:dep_p_ids[j*batch_size:(j+1)*batch_size],
            y:rel_ids[j*batch_size:(j+1)*batch_size]}
        _, _loss, step = sess.run([optimizer, total_loss, global_step], feed_dict)
        loss_per_epoch +=_loss
        if (j+1)%num_batches==0:
            print("Epoch:", i+1,"Step:", step, "loss:",loss_per_epoch/num_batches)
    saver.save(sess, model_dir + '/model')
    print("Saved Model")

Epoch: 1 Step: 800 loss: 2.85300489247
Saved Model
Epoch: 2 Step: 1600 loss: 2.73827668965
Saved Model
Epoch: 3 Step: 2400 loss: 2.70001435518
Saved Model
Epoch: 4 Step: 3200 loss: 2.68624746531
Saved Model
Epoch: 5 Step: 4000 loss: 2.68042603165
Saved Model
Epoch: 6 Step: 4800 loss: 2.67750604913
Saved Model
Epoch: 7 Step: 5600 loss: 2.67583220631
Saved Model
Epoch: 8 Step: 6400 loss: 2.67482194766
Saved Model
Epoch: 9 Step: 7200 loss: 2.67411908716
Saved Model
Epoch: 10 Step: 8000 loss: 2.67369878128
Saved Model


In [ ]:
# training accuracy
all_predictions = []
for j in range(num_batches):
     feed_dict = {
            path_length:path_len[j*batch_size:(j+1)*batch_size],
            word_ids:word_p_ids[j*batch_size:(j+1)*batch_size],
            pos_ids:pos_p_ids[j*batch_size:(j+1)*batch_size],
            dep_ids:dep_p_ids[j*batch_size:(j+1)*batch_size],
            y:rel_ids[j*batch_size:(j+1)*batch_size]}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("training accuracy", accuracy)

In [25]:
f = open('data/test_lca_paths', 'rb')
word_p, dep_p, pos_p = pickle.load(f)
f.close()

relations = []
for line in open('data/test_relations.txt'):
    relations.append(line.strip().split()[0])

length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p[i]):
        word = word.lower()
        word_p[i][j] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p[i]):
        dep_p[i][l] = d if d in dep2id else 'OTH'
        
word_p_ids = np.ones([length, max_len_path],dtype=int)
pos_p_ids = np.ones([length, max_len_path],dtype=int)
dep_p_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path_len = np.array([len(w) for w in word_p], dtype=int)

for i in range(length):
    for j, w in enumerate(word_p[i]):
        word_p_ids[i][j] = word2id[w]
        
    for j, w in enumerate(pos_p[i]):
        pos_p_ids[i][j] = pos_tag(w)
        
    for j, w in enumerate(dep_p[i]):
        dep_p_ids[i][j] = dep2id[w]

# test predictions
all_predictions = []
for j in range(num_batches):
     feed_dict = {
            path_length:path_len[j*batch_size:(j+1)*batch_size],
            word_ids:word_p_ids[j*batch_size:(j+1)*batch_size],
            pos_ids:pos_p_ids[j*batch_size:(j+1)*batch_size],
            dep_ids:dep_p_ids[j*batch_size:(j+1)*batch_size],
            y:rel_ids[j*batch_size:(j+1)*batch_size]}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("test accuracy", accuracy)

ValueError: Cannot feed value of shape (2, 10) for Tensor 'input_1/path1_length:0', which has shape '(10,)'